## AuxTel test LTS-337-003 (Nasmyth rotator)

In this notebook, I slew the Nasmyth2 rotator through it's range of travel, and evaluate range, max slew speed, and accuracy of position.  Desired specs:

| Description | Value       | Unit          |   Name     |
| :---        |    :----:   |       :----:  |       ---: |
|The rotator shall have a minimum range of rotation of:    | ±120       | Degrees   |Aux_Tel_Field_Rotation_Range|
|The rotator shall be able to achieve or surpass this velocity during slews of the telescope.  |3.5       | Degrees/Second      |Aux_Tel_Inst_Rot_Max_Vel|
|The rotator shall have at maximum this absolute angle error.      | 0.01|Degrees|Aux_Tel_Inst_Rot_Abs_Error|

In [1]:
import sys, time, os, asyncio

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient


Bad key "text.kerning_factor" on line 4 in
/opt/lsst/software/stack/conda/miniconda3-py37_4.8.2/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [4]:
# Get EFD client and bring in Lupton's unpacking code
client = EfdClient('summit_efd')

def merge_packed_time_series(packed_dataframe, base_field, stride=1, 
                             ref_timestamp_col="cRIO_timestamp", internal_time_scale="tai"):
    """Select fields that are time samples and unpack them into a dataframe.
            Parameters
            ----------
            packedDF : `pandas.DataFrame`
                packed data frame containing the desired data
            base_field :  `str`
                Base field name that will be expanded to query all
                vector entries.
            stride : `int`, optional
                Only use every stride value when unpacking.  Must be a factor
                of the number of packed values.
                (1 by default)
            ref_timestamp_col : `str`, optional
                Name of the field name to use to assign timestamps to unpacked
                vector fields (default is 'cRIO_timestamp').
            internal_time_scale : `str`, optional
                Time scale to use when converting times to internal formats
                ('tai' by default). Equivalent to EfdClient.internal_scale
        Returns
            -------
            result : `pandas.DataFrame`
                A `pandas.DataFrame` containing the results of the query.
            """
    
    packed_fields = [k for k in packed_dataframe.keys() if k.startswith(base_field)]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID
    npack = len(packed_fields)
    if npack%stride != 0:
        raise RuntimeError(f"Stride must be a factor of the number of packed fields: {stride} v. {npack}")
    packed_len = len(packed_dataframe)
    n_used = npack//stride   # number of raw fields being used
    output = np.empty(n_used*packed_len)
    times = np.empty_like(output, dtype=packed_dataframe[ref_timestamp_col][0])
    
    if packed_len == 1:
        dt = 0
    else:
        dt = (packed_dataframe[ref_timestamp_col][1] - packed_dataframe[ref_timestamp_col][0])/npack
    for i in range(0, npack, stride):
        i0 = i//stride
        output[i0::n_used] = packed_dataframe[f"{base_field}{i}"]
        times[i0::n_used] = packed_dataframe[ref_timestamp_col] + i*dt
     
    timestamps = Time(times, format='unix', scale=internal_time_scale).datetime64
    return pd.DataFrame({base_field:output, "times":times}, index=timestamps)

Starting new HTTPS connection (1): roundtable.lsst.codes:443
https://roundtable.lsst.codes:443 "GET /segwarides/ HTTP/1.1" 200 253
Starting new HTTPS connection (1): roundtable.lsst.codes:443
https://roundtable.lsst.codes:443 "GET /segwarides/creds/summit_efd HTTP/1.1" 200 92


In [5]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.17 sec
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, algorithm)
Read 8 history items for RemoteEvent(ATDomeTrajectory, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, authList)
Read 100 history items for RemoteEvent(ATDomeTrajectory, 0, heartbeat)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, logLevel)
Read 12 history items for RemoteEvent(ATDomeTrajectory, 0, logMessage)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingVersions)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingsApplied)
Read 1 history i

[[None, None, None, None, None, None, None], [None, None, None, None]]

In [6]:
# enable components if required
await atcs.enable()
#await latiss.enable()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for ataos.
Complete settings for atpneumatics.
Complete settings for athexapod.
Complete settings for atdome.
Complete settings for atdometrajectory.
Settings versions: {'atmcs': '                                                                                                                               ', 'atptg': '', 'ataos': 'current', 'atpneumatics': '                                                                                                                               ', 'athexapod': 'summit', 'atdome': 'test', 'atdometrajectory': ''}
[atmcs]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atptg]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atpneumatics]::[<State.STANDBY: 5>, <State.DISABLED: 

RuntimeError: Failed to transition ['atdome'] to <State.ENABLED: 2>.

In [7]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [8]:
# turn on ATAOS corrections just to make sure the mirror is under air
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=False)

In [9]:
# Ensure we're using Nasmyth 2
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [10]:
# slew telescope to desired starting position
# rotator does not move for this test as it's part of a different
# requirement/verification
start_az=0
start_el=80
start_rot_pa=0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

Sending command
Stop tracking.
Unknown tracking state: 10.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Got True
Waiting for telescope to settle.
[Telescope] delta Alt = +000.000 deg; delta Az= -000.000 deg; delta N1 = +000.000 deg; delta N2 = -000.908 deg 
Telescope in position.


In [14]:
# Here is where the tests are carried out.
max_rot = 130.0 # Maximum +/- 130 degrees
rot_step = 10.0 # 10 degree steps
n_steps = int(max_rot / rot_step)
maxes = []
errors = []
speeds = []
signs = [-1.0, 1.0] # Repeat tests in both directions

for sign in signs:
    await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)
    # Move to max_rot
    current_rot = start_rot_pa + sign * max_rot
    await atcs.point_azel(start_az, start_el, rot_tel=current_rot, wait_dome=False)
    current_time = Time(Time.now(), format='fits', scale='tai')
    # Get velocity data for 60 seconds before current time
    # and find median of 100 largest values
    t_end = current_time
    nsec = 60
    nasmyth_velocity = await client.select_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ['*'],
                                              t_end - TimeDelta(nsec, format='sec'), t_end)
    velocity = merge_packed_time_series(nasmyth_velocity, 'nasmyth2MotorVelocity', stride=1)
    velArray = np.array(velocity.values.tolist())[:,0]
    sortedVelArray = velArray[np.argsort(velArray)]
    measuredPeakVelocity = max(abs(np.median(sortedVelArray[0:100])), abs(np.median(sortedVelArray[-100:])))
    print(f"Measured peak velocity = {measuredPeakVelocity}")
    speeds.append(measuredPeakVelocity)
    # Get current position to verify it is beyond the max
    await asyncio.sleep(3)
    current_time = current_time = Time(Time.now(), format='fits', scale='tai')
    t_end = current_time - TimeDelta(1, format='sec')
    nsec = 2
    nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                              t_end - TimeDelta(nsec, format='sec'), t_end)
    angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
    current_angle = angle.values.tolist()[-1][0]
    print(f"Current rotator angle = {current_angle}")
    maxes.append(abs(current_angle))
    # Now slew through a range of angles and evaluate the error
    for n in range(n_steps):
        current_rot = current_rot - sign * rot_step
        await atcs.point_azel(start_az, start_el, rot_tel=current_rot, wait_dome=False)
        # Get current position and compare to set point
        await asyncio.sleep(3)
        current_time = current_time = Time(Time.now(), format='fits', scale='tai')
        t_end = current_time - TimeDelta(1, format='sec')
        nsec = 2
        nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                                  t_end - TimeDelta(nsec, format='sec'), t_end)
        angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
        current_angle = angle.values.tolist()[-1][0]
        error = current_angle - current_rot
        print(f"Current rotator angle = {current_angle}. Set point = {current_rot}.  Error = {error}")
        errors.append(abs(error))


Sending command
Stop tracking.
Unknown tracking state: 9.
Unknown tracking state: 10.
In Position: True.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Got False
Telescope not in position
[Telescope] delta Alt = +000.000 deg; delta Az= -000.000 deg; delta N1 = +000.000 deg; delta N2 = +129.921 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= -000.000 deg; delta N1 = -000.000 deg; delta N2 = +125.123 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= -000.000 deg; delta N1 = -000.000 deg; delta N2 = +119.129 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= -000.000 deg; delta N1 = +000.000 deg; delta N2 = +113.127 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= -000.000 deg; delta N1 = -000.000 deg; delta N2 = +107.121 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= -000.000 deg; delta N1 = -000.000 deg; delta N2 = +103.122

In [15]:
# Now check to see if the specs are met:
Aux_Tel_Field_Rotation_Range = 120.0
if min(maxes) > Aux_Tel_Field_Rotation_Range:
    print(f"Aux_Tel_Field_Rotation_Range passed.  Spec = {Aux_Tel_Field_Rotation_Range}.\
    Measured = {min(maxes)} ")
else:
    print(f"Aux_Tel_Field_Rotation_Range failed!  Spec = {Aux_Tel_Field_Rotation_Range}. \
    Measured = {min(maxes)} ")

Aux_Tel_Inst_Rot_Max_Vel = 3.5
if min(speeds) > Aux_Tel_Inst_Rot_Max_Vel:
    print(f"Aux_Tel_Inst_Rot_Max_Vel passed.  Spec = {Aux_Tel_Inst_Rot_Max_Vel}. \
    Measured = {min(speeds)} ")
else:
    print(f"Aux_Tel_Inst_Rot_Max_Vel failed!  Spec = {Aux_Tel_Inst_Rot_Max_Vel}. \
    Measured = {min(speeds)} ")

Aux_Tel_Inst_Rot_Abs_Error = 0.01
if max(errors) < Aux_Tel_Inst_Rot_Abs_Error:
    print(f"Aux_Tel_Inst_Rot_Abs_Error passed.  Spec = {Aux_Tel_Inst_Rot_Abs_Error}. \
    Worst case error = {max(errors)} ")
else:
    print(f"Aux_Tel_Inst_Rot_Abs_Error failed!  Spec = {Aux_Tel_Inst_Rot_Abs_Error}. \
    Worst case error = {max(errors)} ")


Aux_Tel_Field_Rotation_Range passed.  Spec = 120.0.    Measured = 130.00000542920316 
Aux_Tel_Inst_Rot_Max_Vel failed!  Spec = 3.5.     Measured = 2.1721461324 
Aux_Tel_Inst_Rot_Abs_Error passed.  Spec = 0.01.     Worst case error = 6.780998955946416e-05 


In [16]:
# Putting everything back in standby.
# Need to do it one at a time, since ATDome fails with check=False.
# Need to do this first, but it will fail with atimeout
await atcs.stop_tracking()

Stop tracking.
Unknown tracking state: 9.
Unknown tracking state: 10.
In Position: True.


TimeoutError: 

In [17]:
# Putting everything back in standby.
# Need to do it one at a time, since ATDome fails with check=False.
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]